# Example Noise

Here we demonstrate an example noise analysis

In [1]:
%load_ext autoreload
%autoreload 2

In [43]:
import numpy as np
import pandas as pd

import glob, json, os
import matplotlib.pyplot as plt

from h5pulsar import Pulsar, FilePulsar
from enterprise.signals import parameter
from enterprise.signals import utils
from enterprise.signals import signal_base
from enterprise.signals import selections
from enterprise.signals.selections import Selection
from enterprise.signals import white_signals
from enterprise.signals import gp_signals
from enterprise.signals import deterministic_signals
import enterprise.constants as const
from enterprise.signals import utils

from enterprise_extensions.blocks import common_red_noise_block
from dr3_noise.models import model_singlepsr_noise
from dr3_noise.model_utils import get_freqs
from dr3_noise.selections import CustomSelections
from enterprise_extensions.sampler import setup_sampler, group_from_params
import enterprise.constants as const
from targeted_cws_ng15.Dists_Parameters import get_parameter_groups_CAW_target

%matplotlib inline
%config IPython.matplotlib.backend = "retina"
from matplotlib import rcParams
rcParams["savefig.dpi"] = 300
rcParams["figure.dpi"] = 300

change directories and such to match your usage

In [49]:
project_path = '/vast/palmer/home.grace/bbl29/IPTA_DR2_analysis'
scratchdir = '/vast/palmer/scratch/mingarelli/bbl29/IPTA_DR2_analysis/chains'
datadir = f'{basedir}/data/full_ePSRs'  # par/tim location
outdir = f'{scratchdir}/dr2full/crn_pl13_g4p3'  # chain file and other output here
noisedict_path = f'{project_path}/noisedicts/dr2full_stdnoise.json'
if not os.path.isdir(outdir):
    os.mkdir(outdir)
emp_dist_path = f'{project_path}/empdists/dr2full_stdnoise.pkl'

# read in pulsars

In [7]:
psrnames = []
psr_paths = glob.glob(f'{project_path}/data/full_ePSRs/J*')
for psr_path in psr_paths:
    psrnames.append(psr_path.split('/')[-1][:-5])
psrnames = np.sort(np.unique(psrnames))

In [9]:
psrs = []
for psrname in psrnames:
    psrs.append(FilePulsar(f'{datadir}/{psrname}.hdf5'))

# setup basic enterprise model

This model is intended to correspond to the noise model used for the DR3 GWB paper:
- Timing model: Linear, use `tm_svd`, use `tm_marg`
- White noise: EFACs with Gaussian priors, TNEQUAD, ECORR for NANOGrav only. Fixed values using noise dictionary
- Common red noise: 13-component PL from 1/Tspan
- Red noise: 30-component PL from 1/Tspan
- DMv: 30-component PL from 1/Tspan
- Extra signals: DM dip for J1713 only

We'll set up the model quickly with enterprise_extensions

In [11]:
Tspan = 952746385.6296968

In [20]:
crn = common_red_noise_block(psd='powerlaw', prior='log-uniform', Tspan=Tspan,
                             components=13, gamma_val=13/3,
                             logmin=-18, logmax=-12,
                             orf=None, name='crn')

In [33]:
models = []
for psr in psrs:
    if 'NANOGrav' in psr.flags['pta']:
        inc_ecorr = True
        ecorr_groups_by_PTA = {'NANOGrav':'group'}
    else:
        inc_ecorr = False
        ecorr_groups_by_PTA = None
    efeq_groups_by_PTA = {}
    for pta in np.unique(psr.flags['pta']):
        efeq_groups_by_PTA[pta] = 'group'
    #print(efeq_groups_by_PTA)
    #print(ecorr_groups_by_PTA)
    if psr.name == 'J1713+0747':
        dm_expdip=True
    else:
        dm_expdip=False
    noise = model_singlepsr_noise(psr, Tspan=Tspan, psr_model=True,
                                  # timing
                                  tm_svd=True, tm_marg=True,
                                  # white noise
                                  white_vary=False, tnequad=True, inc_ecorr=inc_ecorr,
                                  efeq_groups=efeq_groups_by_PTA, ecorr_groups=ecorr_groups_by_PTA,
                                  log_equad_min=-10, log_equad_max=-4,
                                  # DM
                                  dm_var=True, dm_type='gp',
                                  dmgp_kernel='diag', dm_psd='powerlaw',
                                  dm_Nfreqs=30,
                                  # solar wind
                                  dm_sw_deter=False,
                                  # dm dip
                                  dm_expdip=dm_expdip, dm_expdip_basename='exp',
                                  dm_expdip_tau_min=np.log10(5), dm_expdip_tau_max=np.log10(500), 
                                  # red noise
                                  log_A_min=-20, log_A_max=-11)
    signals = crn + noise
    models.append(signals(psr))
pta = signal_base.PTA(models)
with open(noisedict_path,'r') as f:
    noise_params = json.load(f)
pta.set_default_params(noise_params)

In [34]:
print(pta.summary())

enterprise v0.1.dev796+gcb3e25e,  Python v3.10.12

Signal Name                              Signal Class                   no. Parameters      
J0030+0451_crn                           FourierBasisGP                 1                   

Basis shape (Ntoas x N basis functions): (3362, 26)
N selected toas: 3362

params:
crn_log10_A:Uniform(pmin=-18, pmax=-12)                                                   
crn_gamma:Constant=4.333333333333333                                                      
__________________________________________________________________________________________
J0030+0451_marginalizing_linear_timing_model TimingModel                    0                   

params:
__________________________________________________________________________________________
J0030+0451_red_noise                     FourierBasisGP                 2                   

Basis shape (Ntoas x N basis functions): (3362, 60)
N selected toas: 3362

params:
J0030+0451_red_noise_log10_A:Uni

In [35]:
def likelihood_eval(pta):
    random_sample = {p.name:p.sample() for p in pta.params}
    return pta.get_lnlikelihood(random_sample)

In [36]:
%timeit likelihood_eval(pta)

18.9 ms ± 553 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [48]:
# set groups for adaptive metropolis
gr = get_parameter_groups_CAW_target(pta)
for group in gr:
    print(np.array(pta.param_names)[np.array(group)])

test
['J0030+0451_dm_gp_gamma' 'J0030+0451_dm_gp_log10_A'
 'J0030+0451_red_noise_gamma' 'J0030+0451_red_noise_log10_A'
 'J0034-0534_dm_gp_gamma' 'J0034-0534_dm_gp_log10_A'
 'J0034-0534_red_noise_gamma' 'J0034-0534_red_noise_log10_A'
 'J0218+4232_dm_gp_gamma' 'J0218+4232_dm_gp_log10_A'
 'J0218+4232_red_noise_gamma' 'J0218+4232_red_noise_log10_A'
 'J0437-4715_dm_gp_gamma' 'J0437-4715_dm_gp_log10_A'
 'J0437-4715_red_noise_gamma' 'J0437-4715_red_noise_log10_A'
 'J0610-2100_dm_gp_gamma' 'J0610-2100_dm_gp_log10_A'
 'J0610-2100_red_noise_gamma' 'J0610-2100_red_noise_log10_A'
 'J0613-0200_dm_gp_gamma' 'J0613-0200_dm_gp_log10_A'
 'J0613-0200_red_noise_gamma' 'J0613-0200_red_noise_log10_A'
 'J0621+1002_dm_gp_gamma' 'J0621+1002_dm_gp_log10_A'
 'J0621+1002_red_noise_gamma' 'J0621+1002_red_noise_log10_A'
 'J0711-6830_dm_gp_gamma' 'J0711-6830_dm_gp_log10_A'
 'J0711-6830_red_noise_gamma' 'J0711-6830_red_noise_log10_A'
 'J0751+1807_dm_gp_gamma' 'J0751+1807_dm_gp_log10_A'
 'J0751+1807_red_noise_gamma' 

In [51]:
# check sampler is setup correctly
if emp_dist_path is not None:
    print(f"Using empirical distributions file path: {emp_dist_path}")
else:
    print("No empirical distributions file given")
sampler = setup_sampler(pta, outdir=outdir, groups=gr, resume=False,
                        human='bjorn.larsen', empirical_distr=emp_dist_path)

Using empirical distributions file path: /vast/palmer/home.grace/bbl29/IPTA_DR2_analysis/empdists/dr2full_stdnoise.pkl
extending empirical distributions to priors...

Attempting to add empirical proposals...

Adding red noise prior draws...

Adding DM GP noise prior draws...

